In [1]:
import pandas as pd
import sqlite3 as sql

In [2]:
conn = sql.connect("C:\sql-murder-mystery.db")

In [3]:
conn

Query 1:
    Retrive crime scene report

In [6]:
# running our first query, looking for data from the crime scene report
query_1 = '''
SELECT *
FROM crime_scene_report
WHERE city = "SQL City"
ORDER BY date;
'''

# setting the dataframe width to max columns
pd.set_option('display.max_columns', None)

# running our query and displaying the results in a DataFrame
df = pd.read_sql_query(query_1, conn)
print(df)


       date     type                                        description  \
0  20170712    theft  A lone hunter stalks the night, firing arrows ...   
1  20170820    arson  Wield the Hammer of Sol with honor, Titan, it ...   
2  20171110  robbery  The Gjallarhorn shoulder-mounted rocket system...   
3  20180103  bribery  Apparently, Cayde thought it necessary to expo...   
4  20180115  assault  Hamilton: Lee, do you yield? Burr: You shot hi...   
5  20180115  assault                                   Report Not Found   
6  20180115   murder  Security footage shows that there were 2 witne...   
7  20180215   murder                         REDACTED REDACTED REDACTED   
8  20180215   murder  Someone killed the guard! He took an arrow to ...   

       city  
0  SQL City  
1  SQL City  
2  SQL City  
3  SQL City  
4  SQL City  
5  SQL City  
6  SQL City  
7  SQL City  
8  SQL City  


Murder occured on 15th Jan-2018, Security footage shows that there were 2 witnesses

Witness 1: lives on last house on "Northwestern Dr"
Witness 2: lives on Somewhere on "Franklin Ave" Name= "Annabel"

Query 2: Witness personal Details

In [7]:
# checking personal details of both witnesses
query_2 = '''
SELECT *
FROM person
WHERE address_street_name = "Northwestern Dr"
ORDER BY address_number DESC
LIMIT 1;
'''

# running the query and displaying the result for the first row
witness_details = pd.read_sql_query(query_2, conn)
print(witness_details)


      id            name  license_id  address_number address_street_name  \
0  14887  Morty Schapiro      118009            4919     Northwestern Dr   

         ssn  
0  111564949  


First Witness Name = "Morty Schapiro"

In [8]:
Q2_2 = '''
SELECT *
FROM person
WHERE address_street_name = 'Franklin Ave'
AND name LIKE '%Annabel%';
'''
pd.read_sql_query(Q2_2,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


Second Witness Name: "Annabel Miller"

Query 3: View witness Interviews

In [10]:
Query_3 = '''
SELECT *
FROM interview
WHERE person_id IN (14887,16371);
'''
pd.read_sql_query(Query_3, conn)

,person_id,transcript
0,14887,I heard a gunshot and then saw a man run out. ...
1,16371,"I saw the murder happen, and I recognized the ..."


Clues:
1.Member of Get Fit Now Gym
2.Membership status = 'Gold'
3.Car plate includes 'H42W'

Query 4: Check Gym Database

In [11]:
Query_4_1 = '''
SELECT *
FROM  get_fit_now_member
WHERE id LIKE '48Z%' AND membership_status = 'gold';
'''
pd.read_sql_query(Query_4_1,conn)

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


Discovered two people who have the hints mentioned below

Query 5: Check Car details

In [13]:
Query_5 = '''
SELECT *
FROM drivers_license
WHERE plate_number  LIKE '%H42W%';
'''
pd.read_sql_query(Query_5,conn)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,664760,21,71,black,black,male,4H42WR,Nissan,Altima


Three car's has number plate which includes "H42W"

Query 6: Personal Details

In [15]:
Query_6 = '''
SELECT *
FROM person
WHERE license_id IN (183779, 423327, 664760);
'''
pd.read_sql_query(Query_6,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
2,78193,Maxine Whitely,183779,110,Fisk Rd,137882671


Retrived the personal details of car owners

Query 7: Membership status at the Gym

In [16]:
Query_7 = '''
SELECT *
FROM get_fit_now_member
WHERE person_id IN (51739,67318,78193);
'''
pd.read_sql_query(Query_7,conn)

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold


Discovered the murderer: "Jeremy Bowers"

In [18]:
# Reading the transcript of the murderer from the interview
query_8 = '''
SELECT *
FROM interview
WHERE person_id = "67318";
'''


murderer_transcript = pd.read_sql_query(query_8, conn)
print(murderer_transcript)

   person_id                                         transcript
0      67318  I was hired by a woman with a lot of money. I ...


Interviewed the murderer and discovered that a mastermind was hiding behind him 
Hints:
1.Gender: female
2.Hair_color: red
3.Height between 65 and 67
4.Car make: Tesla
5.Car_model: Model S
6.Attended facebook event 'Symphony Concert' 3 times in December 2017

In [19]:
Query_8_2 = '''
SELECT p.*, dl.car_make, dl.car_model,dl.hair_color, dl.height
FROM person as p
INNER JOIN drivers_license as dl on dl.id = p.license_id
INNER JOIN facebook_event_checkin as fb_event on p.id = fb_event.person_id
WHERE car_make = 'Tesla' AND car_model = 'Model S' AND hair_color = 'red' AND height BETWEEN 65 AND 67;
'''
pd.read_sql_query(Query_8_2,conn)

,id,name,license_id,address_number,address_street_name,ssn,car_make,car_model,hair_color,height
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,Tesla,Model S,red,66
1,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,Tesla,Model S,red,66
2,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,Tesla,Model S,red,66


Found the Mastermind behind this murder: "Miranda Priestly"

In [20]:
print("You've been tirelessly investigating the case for weeks.")
print("Suddenly, a revelation is on the horizon...")

# Pause for dramatic effect
input("Press Enter to continue...")

print("The room plunges into darkness as a mysterious figure steps forward.")
print("It's Miranda Priestly, the mastermind you never suspected!")

# Continue the story
print("Miranda: 'You believed you were in control, but little did you know, you were just a pawn.'")
print("As Miranda unveils her nefarious plans, the true extent of the conspiracy becomes clear.")

# Conclude the story
print("In the final showdown, justice prevails, and the enigma is ultimately unraveled.")
print("The end.")


You've been tirelessly investigating the case for weeks.
Suddenly, a revelation is on the horizon...
Press Enter to continue...
The room plunges into darkness as a mysterious figure steps forward.
It's Miranda Priestly, the mastermind you never suspected!
Miranda: 'You believed you were in control, but little did you know, you were just a pawn.'
As Miranda unveils her nefarious plans, the true extent of the conspiracy becomes clear.
In the final showdown, justice prevails, and the enigma is ultimately unraveled.
The end.
